# Monthly netcdfs for models

In [2]:
# %% import modules
import sys
sys.path.append('../')  # import functions to make life easier
import climate_xr_funcs
import climate_droughts_xr_funcs

# import things
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cftime
import regionmask
import os
from dask.diagnostics import ProgressBar

from scipy import stats

In [4]:
def get_aus(ds):
    mask = regionmask.defined_regions.natural_earth.countries_110.mask(ds)
    ds_aus = ds.where(mask == 137, drop=True)
    return ds_aus

In [5]:
def read_in_pmip3(modelname, var):
    if modelname == 'bcc':
        ds = xr.open_mfdataset('%s/past1000/%s/%s_Amon_bcc-csm1-1_past1000_r1i1p1_*.nc' % (filepath_pmip3, var, var),
                               combine='by_coords', chunks={'time': 1000})
    if modelname == 'ccsm4':
        ds_p1 = xr.open_dataset('%s/past1000/%s/%s_Amon_CCSM4_past1000_r1i1p1_085001-185012.nc' % (filepath_pmip3, var, var),
                                chunks={'time': 1000})
        ds_p2 = xr.open_dataset('%s/historical/%s/%s_Amon_CCSM4_historical_r1i1p1_185001-200512.nc' % (filepath_pmip3, var, var),
                                chunks={'time': 1000})
        ds_p2 = ds_p2.where(ds_p2.time >= cftime.DatetimeNoLeap(1851,1,1), drop=True)  # get rid of duplicate 1850 year
        ds_p2['lat'] = ds_p1.lat
        ds = xr.concat([ds_p1, ds_p2], dim='time')
    if modelname == 'csiro_mk3l':
        ds_p1 = xr.open_dataset('%s/past1000/%s/%s_Amon_CSIRO-Mk3L-1-2_past1000_r1i1p1_085101-185012.nc' % (filepath_pmip3, var, var), chunks={'time': 1000})
        ds_p2 = xr.open_dataset('%s/historical/%s/%s_Amon_CSIRO-Mk3L-1-2_historical_r1i1p1_185101-200012.nc' % (filepath_pmip3, var, var), chunks={'time': 1000})
        ds = xr.concat([ds_p1, ds_p2], dim='time')
    if modelname == 'fgoals_gl':
        ds = xr.open_dataset('%s/past1000/%s/%s_Amon_FGOALS-gl_past1000_r1i1p1_100001-199912.nc'  % (filepath_pmip3, var, var), chunks={'time': 1000})
    if modelname == 'fgoals_s2':
        if var =='ts':
            ds_p1 = xr.open_mfdataset('%s/past1000/%s/%s_Amon_FGOALS-s2_past1000_r1i1p1_*.nc'  % (filepath_pmip3, var, var), combine='by_coords', chunks={'time': 1000})
            ds_p2 = xr.open_dataset('%s/historical/%s/%s_Amon_FGOALS-s2_historical_r1i1p1_185001-200512.nc'  % (filepath_pmip3, var, var), chunks={'time': 1000})
            ds_p2 = ds_p2.where(ds_p2.time >= cftime.DatetimeNoLeap(1851,1,1), drop=True)  # get rid of duplicate 1850 year
            ds = xr.concat([ds_p1, ds_p2], dim='time')
        elif var =='pr':
            ds_p1 = xr.open_mfdataset('%s/past1000/%s/%s_Amon_FGOALS-s2_past1000_r1i1p1_*.nc'  % (filepath_pmip3, var, var), combine='by_coords', chunks={'time': 1000})
            ds_p2 = xr.open_dataset('%s/historical/%s/%s_Amon_FGOALS-s2_historical_r1i1p1_185001-200512.nc'  % (filepath_pmip3, var, var), chunks={'time': 1000})
            ds_p2 = ds_p2.where(ds_p2.time >= cftime.DatetimeNoLeap(1851,1,1), drop=True)  # get rid of duplicate 1850 year
            ds = xr.concat([ds_p1, ds_p2], dim='time')
        else:
            ds_p1 = xr.open_dataset('%s/past1000/%s/%s_Amon_FGOALS-s2_past1000_r1i1p1_085001-185012.nc'  % (filepath_pmip3, var, var), chunks={'time': 1000})
            ds_p2 = xr.open_dataset('%s/historical/%s/%s_Amon_FGOALS-s2_historical_r1i1p1_185001-200512.nc'  % (filepath_pmip3, var, var), chunks={'time': 1000})
            ds_p2 = ds_p2.where(ds_p2.time >= cftime.DatetimeNoLeap(1851,1,1), drop=True)  # get rid of duplicate 1850 year
            ds = xr.concat([ds_p1, ds_p2], dim='time')
    if modelname == 'giss_21':
        if var == 'ts' or var == 'pr':
            ds_p1 = xr.open_mfdataset('%s/past1000/%s/%s_Amon_GISS-E2-R_past1000_r1i1p121_*.nc'  % (filepath_pmip3, var, var),  combine='by_coords', chunks={'time': 600})
            ds_p2 = xr.open_mfdataset('%s/historical/%s/%s_Amon_GISS-E2-R_historical_r1i1p121_*.nc' % (filepath_pmip3, var, var), combine='by_coords', chunks={'time': 600})
            ds = xr.concat([ds_p1, ds_p2], dim='time')
        else:
            ds_p1 = xr.open_dataset('%s/past1000/%s/%s_Amon_GISS-E2-R_past1000_r1i1p121_085001-185012.nc'  % (filepath_pmip3, var, var), chunks={'time': 600})
            ds_p2 = xr.open_mfdataset('%s/historical/%s/%s_Amon_GISS-E2-R_historical_r1i1p121_*.nc' % (filepath_pmip3, var, var), combine='by_coords', chunks={'time': 600})
            ds = xr.concat([ds_p1, ds_p2], dim='time')
    if modelname == 'giss_22':
        if var == 'ts' or var == 'pr':
            ds_p1 = xr.open_mfdataset('%s/past1000/%s/%s_Amon_GISS-E2-R_past1000_r1i1p122_*.nc'  % (filepath_pmip3, var, var),  combine='by_coords', chunks={'time': 600})
            ds_p2 = xr.open_mfdataset('%s/historical/%s/%s_Amon_GISS-E2-R_historical_r1i1p122_*.nc' % (filepath_pmip3, var, var), combine='by_coords', chunks={'time': 600})
            ds = xr.concat([ds_p1, ds_p2], dim='time')
        else:
            ds_p1 = xr.open_dataset('%s/past1000/%s/%s_Amon_GISS-E2-R_past1000_r1i1p122_085001-185012.nc'  % (filepath_pmip3, var, var), chunks={'time': 600})
            ds_p2 = xr.open_mfdataset('%s/historical/%s/%s_Amon_GISS-E2-R_historical_r1i1p122_*.nc' % (filepath_pmip3, var, var), combine='by_coords', chunks={'time': 600})
            ds = xr.concat([ds_p1, ds_p2], dim='time')
    if modelname == 'giss_23':
        if var == 'ts' or var == 'pr':
            ds_p1 = xr.open_mfdataset('%s/past1000/%s/%s_Amon_GISS-E2-R_past1000_r1i1p123_*.nc'  % (filepath_pmip3, var, var),  combine='by_coords', chunks={'time': 600})
            ds_p2 = xr.open_mfdataset('%s/historical/%s/%s_Amon_GISS-E2-R_historical_r1i1p123_*.nc' % (filepath_pmip3, var, var), combine='by_coords', chunks={'time': 600})
            ds = xr.concat([ds_p1, ds_p2], dim='time')
        else:
            ds_p1 = xr.open_dataset('%s/past1000/%s/%s_Amon_GISS-E2-R_past1000_r1i1p123_085001-185012.nc'  % (filepath_pmip3, var, var), chunks={'time': 600})
            ds_p2 = xr.open_mfdataset('%s/historical/%s/%s_Amon_GISS-E2-R_historical_r1i1p123_*.nc' % (filepath_pmip3, var, var), combine='by_coords', chunks={'time': 600})
            ds = xr.concat([ds_p1, ds_p2], dim='time')
    if modelname == 'giss_24':
        if var == 'ts' or var == 'pr':
            ds_p1 = xr.open_mfdataset('%s/past1000/%s/%s_Amon_GISS-E2-R_past1000_r1i1p124_*.nc'  % (filepath_pmip3, var, var),  combine='by_coords', chunks={'time': 600})
            ds_p2 = xr.open_mfdataset('%s/historical/%s/%s_Amon_GISS-E2-R_historical_r1i1p124_*.nc' % (filepath_pmip3, var, var), combine='by_coords', chunks={'time': 600})
            ds = xr.concat([ds_p1, ds_p2], dim='time')
        else:
            ds_p1 = xr.open_dataset('%s/past1000/%s/%s_Amon_GISS-E2-R_past1000_r1i1p124_085001-185012.nc'  % (filepath_pmip3, var, var), chunks={'time': 600})
            ds_p2 = xr.open_mfdataset('%s/historical/%s/%s_Amon_GISS-E2-R_historical_r1i1p124_*.nc' % (filepath_pmip3, var, var), combine='by_coords', chunks={'time': 600})
            ds = xr.concat([ds_p1, ds_p2], dim='time')
    if modelname == 'giss_25':
        if var == 'ts' or var == 'pr':
            ds_p1 = xr.open_mfdataset('%s/past1000/%s/%s_Amon_GISS-E2-R_past1000_r1i1p125_*.nc'  % (filepath_pmip3, var, var),  combine='by_coords', chunks={'time': 600})
            ds_p2 = xr.open_mfdataset('%s/historical/%s/%s_Amon_GISS-E2-R_historical_r1i1p125_*.nc' % (filepath_pmip3, var, var), combine='by_coords', chunks={'time': 600})
            ds = xr.concat([ds_p1, ds_p2], dim='time')
        else:
            ds_p1 = xr.open_dataset('%s/past1000/%s/%s_Amon_GISS-E2-R_past1000_r1i1p125_085001-185012.nc'  % (filepath_pmip3, var, var), chunks={'time': 600})
            ds_p2 = xr.open_mfdataset('%s/historical/%s/%s_Amon_GISS-E2-R_historical_r1i1p125_*.nc' % (filepath_pmip3, var, var), combine='by_coords', chunks={'time': 600})
            ds = xr.concat([ds_p1, ds_p2], dim='time')
    if modelname == 'giss_26':
        if var == 'ts' or var == 'pr':
            ds_p1 = xr.open_mfdataset('%s/past1000/%s/%s_Amon_GISS-E2-R_past1000_r1i1p126_*.nc'  % (filepath_pmip3, var, var),  combine='by_coords', chunks={'time': 600})
            ds_p2 = xr.open_mfdataset('%s/historical/%s/%s_Amon_GISS-E2-R_historical_r1i1p126_*.nc' % (filepath_pmip3, var, var), combine='by_coords', chunks={'time': 600})
            ds = xr.concat([ds_p1, ds_p2], dim='time')
        else:
            ds_p1 = xr.open_dataset('%s/past1000/%s/%s_Amon_GISS-E2-R_past1000_r1i1p126_085001-185012.nc'  % (filepath_pmip3, var, var), chunks={'time': 600})
            ds_p2 = xr.open_mfdataset('%s/historical/%s/%s_Amon_GISS-E2-R_historical_r1i1p126_*.nc' % (filepath_pmip3, var, var), combine='by_coords', chunks={'time': 600})
            ds = xr.concat([ds_p1, ds_p2], dim='time')
    if modelname == 'giss_27':
        if var == 'ts' or var == 'pr':
            ds_p1 = xr.open_mfdataset('%s/past1000/%s/%s_Amon_GISS-E2-R_past1000_r1i1p127_*.nc'  % (filepath_pmip3, var, var),  combine='by_coords', chunks={'time': 600})
            ds_p2 = xr.open_mfdataset('%s/historical/%s/%s_Amon_GISS-E2-R_historical_r1i1p127_*.nc' % (filepath_pmip3, var, var), combine='by_coords', chunks={'time': 600})
            ds = xr.concat([ds_p1, ds_p2], dim='time')
        else:
            ds_p1 = xr.open_dataset('%s/past1000/%s/%s_Amon_GISS-E2-R_past1000_r1i1p127_085001-185012.nc'  % (filepath_pmip3, var, var), chunks={'time': 600})
            ds_p2 = xr.open_mfdataset('%s/historical/%s/%s_Amon_GISS-E2-R_historical_r1i1p127_*.nc' % (filepath_pmip3, var, var), combine='by_coords', chunks={'time': 600})
            ds = xr.concat([ds_p1, ds_p2], dim='time')
    if modelname == 'giss_28':
        if var == 'ts' or var == 'pr' or var == 'psl':
            ds_p1 = xr.open_mfdataset('%s/past1000/%s/%s_Amon_GISS-E2-R_past1000_r1i1p128_*.nc'  % (filepath_pmip3, var, var),  combine='by_coords', chunks={'time': 600})
            ds_p2 = xr.open_mfdataset('%s/historical/%s/%s_Amon_GISS-E2-R_historical_r1i1p128_*.nc' % (filepath_pmip3, var, var), combine='by_coords', chunks={'time': 600})
            ds = xr.concat([ds_p1, ds_p2], dim='time')
        else:
            ds_p1 = xr.open_dataset('%s/past1000/%s/%s_Amon_GISS-E2-R_past1000_r1i1p128_085001-185012.nc'  % (filepath_pmip3, var, var), chunks={'time': 600})
            ds_p2 = xr.open_mfdataset('%s/historical/%s/%s_Amon_GISS-E2-R_historical_r1i1p128_*.nc' % (filepath_pmip3, var, var), combine='by_coords', chunks={'time': 600})
            ds = xr.concat([ds_p1, ds_p2], dim='time')
    if modelname == 'hadcm3':
        print('Friendly reminder that hadcm3 is missing years 1801-1859...')
        ds_p1 = xr.open_dataset('%s/past1000/%s/%s_Amon_HadCM3_past1000_r1i1p1_085001-185012.nc'  % (filepath_pmip3, var, var), chunks={'time': 600})
        ds_p2 = xr.open_mfdataset('%s/historical/%s/%s_Amon_HadCM3_historical_r1i1p1_*.nc'  % (filepath_pmip3, var, var), combine='by_coords', chunks={'time': 600})
        ds = xr.concat([ds_p1, ds_p2], dim='time')
        # something weird is happening with the time field, re-read it to be consistent just in case
        new_times = cftime.date2num(ds.time, calendar='365_day', units='days since 850-01-01')
        attrs = {'units': 'days since 850-01-01 00:00:00', 'calendar': '365_day'}
        dates = xr.Dataset({'time': ('time', new_times, attrs)})
        dates = xr.decode_cf(dates)
        # ds.update({'time':('time', dates['time'], attrs)})
        ds['time'] = dates['time']

    if modelname == 'ipsl':
        ds_p1 = xr.open_dataset('%s/past1000/%s/%s_Amon_IPSL-CM5A-LR_past1000_r1i1p1_085001-185012.nc' % (filepath_pmip3, var, var), chunks={'time': 600})
        ds_p2 = xr.open_dataset('%s/historical/%s/%s_Amon_IPSL-CM5A-LR_historical_r1i1p1_185001-200512.nc' % (filepath_pmip3, var, var), chunks={'time': 600})
        ds_p2 = ds_p2.where(ds_p2.time >= cftime.DatetimeNoLeap(1851,1,1), drop=True)  # get rid of duplicate 1850 year
        ds = xr.concat([ds_p1, ds_p2], dim='time')

        # something weird is happening with the time field, re-read it to be consistent just in case
        new_times = cftime.date2num(ds.time, calendar='365_day', units='days since 850-01-01')
        attrs = {'units': 'days since 850-01-01 00:00:00', 'calendar': '365_day'}
        dates = xr.Dataset({'time': ('time', new_times, attrs)})
        dates = xr.decode_cf(dates)
        # ds.update({'time':('time', dates['time'], attrs)})
        ds['time'] = dates['time']

    if modelname == 'miroc':
        ds_p1 = xr.open_dataset('%s/past1000/%s/%s_Amon_MIROC-ESM_past1000_r1i1p1_085001-184912.nc' % (filepath_pmip3, var, var), chunks={'time': 600})
        ds_p2 = xr.open_dataset('%s/historical/%s/%s_Amon_MIROC-ESM_historical_r1i1p1_185001-200512.nc' % (filepath_pmip3, var, var), chunks={'time': 600})
        ds = xr.concat([ds_p1, ds_p2], dim='time')

        # fix times
        new_times = cftime.date2num(ds.time, calendar='365_day', units='days since 850-01-01')
        attrs = {'units': 'days since 850-01-01 00:00:00', 'calendar': '365_day'}
        dates = xr.Dataset({'time': ('time', new_times, attrs)})
        dates = xr.decode_cf(dates)
        # ds.update({'time':('time', dates['time'], attrs)})
        ds['time'] = dates['time']

    if modelname == 'mpi':
        ds_p1 = xr.open_dataset('%s/past1000/%s/%s_Amon_MPI-ESM-P_past1000_r1i1p1_085001-184912.nc' % (filepath_pmip3, var, var), chunks={'time': 600})
        ds_p2 = xr.open_dataset('%s/historical/%s/%s_Amon_MPI-ESM-P_historical_r1i1p1_185001-200512.nc' % (filepath_pmip3, var, var), chunks={'time': 600})
        ds = xr.concat([ds_p1, ds_p2], dim='time')

        # fix times
        new_times = cftime.date2num(ds.time, calendar='365_day', units='days since 850-01-01')
        attrs = {'units': 'days since 850-01-01 00:00:00', 'calendar': '365_day'}
        dates = xr.Dataset({'time': ('time', new_times, attrs)})
        dates = xr.decode_cf(dates)
        # ds.update({'time':('time', dates['time'], attrs)})
        ds['time'] = dates['time']

    if modelname == 'mri':
        ds_p1 = xr.open_dataset('%s/past1000/%s/%s_Amon_MRI-CGCM3_past1000_r1i1p1_085001-185012.nc' % (filepath_pmip3, var, var), chunks={'time': 600})
        ds_p2 = xr.open_dataset('%s/historical/%s/%s_Amon_MRI-CGCM3_historical_r1i1p1_185001-200512.nc' % (filepath_pmip3, var, var), chunks={'time': 600}, decode_times=False)
        # try and fix the times...
        newdates = cftime.num2date(ds_p2.time.values, 'days since 1850-01-01',  calendar='standard')
        attrs = {'units': 'days since 850-01-01 00:00:00', 'calendar': '365_day'}# does this make sense?
        dates = xr.Dataset({'time': ('time', newdates, attrs)})
        # dates = xr.decode_cf(dates)
        # ds_p2.update({'time':('time', dates['time'], attrs)})
        ds_p2['time'] = dates['time']

        ds_p2 = ds_p2.where(ds_p2.time >= cftime.DatetimeGregorian(1851,1,1), drop=True)
        ds = xr.concat([ds_p1, ds_p2], dim='time')

        # fix times again
        new_times = cftime.date2num(ds.time, calendar='365_day', units='days since 850-01-01')
        attrs = {'units': 'days since 850-01-01 00:00:00', 'calendar': '365_day'}
        dates = xr.Dataset({'time': ('time', new_times, attrs)})
        dates = xr.decode_cf(dates)
        # ds.update({'time':('time', dates['time'], attrs)})
        ds['time'] = dates['time']        

    return ds

In [10]:
def save_netcdf_compression(ds, output_dir, filename):

    comp = dict(zlib=True, complevel=8)
    encoding = {var: comp for var in ds.data_vars}

    delayed_obj = ds.to_netcdf('%s/%s.nc' % (output_dir, filename), mode='w', compute=False, encoding=encoding)
    with ProgressBar():
        results = delayed_obj.compute()


In [39]:
def convert_PMIP3_to_PRECT_mm_and_get_Aus(ds, output_dir, filename):
    # import monthly pmip3 and run drought workflow
    # convert monthly to annual
    month_length = xr.DataArray(climate_xr_funcs.get_dpm(ds.pr, calendar='noleap'), coords=[ds.pr.time], name='month_length')
    ds['PRECT_mm'] = ds.pr * 60 * 60 * 24 * month_length # to be in mm/month first
    
    ds_aus = get_aus(ds)
    
    try:
        ds_aus2 = ds_aus.drop(['time_bnds'])
    except ValueError:
        ds_aus2 = ds_aus
    save_netcdf_compression(ds_aus2, output_dir, filename)
    
    return ds_aus2

## Paths and output dir

In [3]:
# %% set file path to model output
filepath = '/Volumes/LaCie/CMIP5-PMIP3/CESM-LME/mon/PRECT_v6/'
filepath_pmip3 = '/Volumes/LaCie/CMIP5-PMIP3'
filepath_cesm_mon = '/Volumes/LaCie/CMIP5-PMIP3/CESM-LME/mon'


In [13]:
output_dir = '../files/hist_precip_monthly_aus'
if not os.path.exists(output_dir):
    os.mkdir(output_dir)

## Import files

In [11]:
bcc_pr = read_in_pmip3('bcc', 'pr')
ccsm4_pr  = read_in_pmip3('ccsm4', 'pr')
csiro_mk3l_pr  = read_in_pmip3('csiro_mk3l', 'pr')
fgoals_gl_pr  = read_in_pmip3('fgoals_gl', 'pr')
fgoals_s2_pr  = read_in_pmip3('fgoals_s2', 'pr')
giss_21_pr  = read_in_pmip3('giss_21', 'pr')
giss_22_pr  = read_in_pmip3('giss_22', 'pr')
giss_23_pr  = read_in_pmip3('giss_23', 'pr')
giss_24_pr  = read_in_pmip3('giss_24', 'pr')
giss_25_pr  = read_in_pmip3('giss_25', 'pr')
giss_26_pr  = read_in_pmip3('giss_26', 'pr')
giss_27_pr  = read_in_pmip3('giss_27', 'pr')
giss_28_pr  = read_in_pmip3('giss_28', 'pr')
hadcm3_pr  = read_in_pmip3('hadcm3', 'pr')
ipsl_pr  = read_in_pmip3('ipsl', 'pr')
miroc_pr  = read_in_pmip3('miroc', 'pr')
mpi_pr  = read_in_pmip3('mpi', 'pr')
mri_pr  = read_in_pmip3('mri', 'pr')

# fix some time issues
ccsm4_pr['time'] = giss_21_pr.time
hadcm3_pr = hadcm3_pr.resample(time='MS').mean()  # resample so we have nans were needed
hadcm3_pr['time'] = giss_21_pr.time

Friendly reminder that hadcm3 is missing years 1801-1859...


/Users/nickywright/miniconda3/envs/py38-x86/lib/python3.8/site-packages/xarray/coding/times.py:144: SerializationWarning: Ambiguous reference date string: 850-1-1. The first value is assumed to be the year hence will be padded with zeros to remove the ambiguity (the padded reference date string is: 0850-1-1). To remove this message, remove the ambiguity by padding your reference date strings with zeros.
  warnings.warn(warning_msg, SerializationWarning)
/Users/nickywright/miniconda3/envs/py38-x86/lib/python3.8/site-packages/xarray/coding/times.py:673: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/Users/nickywright/miniconda3/envs/py38-x86/lib/python3.8/site-packages/xarray/core/indexing.py:423: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using c

In [15]:
ccsm4_pr

<xarray.Dataset>
Dimensions:    (time: 13872, bnds: 2, lat: 192, lon: 288)
Coordinates:
  * time       (time) object 0850-01-16 12:00:00 ... 2005-12-16 12:00:00
  * lat        (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * lon        (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) object dask.array<chunksize=(1000, 2), meta=np.ndarray>
    lat_bnds   (lat, bnds, time) float64 dask.array<chunksize=(192, 2, 12012), meta=np.ndarray>
    lon_bnds   (lon, bnds, time) float64 dask.array<chunksize=(288, 2, 12012), meta=np.ndarray>
    pr         (time, lat, lon) float32 dask.array<chunksize=(1000, 192, 288), meta=np.ndarray>
Attributes: (12/35)
    institution:                  NCAR (National Center for Atmospheric Resea...
    institute_id:                 NCAR
    experiment_id:                past1000
    source:                       CCSM4
    model_id:                     CCSM4
    forcing:                      Sl GHG Vl LU (time-varying over course of s...
    ...                           ...
    table_id:                     Table Amon (26 July 2011) 976b7fd1d9e1be31d...
    title:                        CCSM4 model output prepared for CMIP5 last ...
    parent_experiment:            N/A
    modeling_realm:               atmos
    realization:                  1
    cmor_version:                 2.7.1

In [41]:
# subset to Australia only
# bcc_precip_hist_aus = convert_PMIP3_to_PRECT_mm_and_get_Aus(bcc_pr, output_dir, 'bcc_precip_hist_aus')
# ccsm4_precip_hist_aus = convert_PMIP3_to_PRECT_mm_and_get_Aus(ccsm4_pr, output_dir, 'ccsm4_precip_hist_aus')
# csiro_mk3l_precip_hist_aus = convert_PMIP3_to_PRECT_mm_and_get_Aus(csiro_mk3l_pr, output_dir, 'csiro_mk3l_precip_hist_aus')
# fgoals_gl_precip_hist_aus = convert_PMIP3_to_PRECT_mm_and_get_Aus(fgoals_gl_pr, output_dir, 'fgoals_gl_precip_hist_aus')
# fgoals_s2_precip_hist_aus = convert_PMIP3_to_PRECT_mm_and_get_Aus(fgoals_s2_pr,  output_dir, 'fgoals_s2_precip_hist_aus')

# giss_21_precip_hist_aus = convert_PMIP3_to_PRECT_mm_and_get_Aus(giss_21_pr, output_dir, 'giss_21_precip_hist_aus')
# giss_22_precip_hist_aus = convert_PMIP3_to_PRECT_mm_and_get_Aus(giss_22_pr, output_dir, 'giss_22_precip_hist_aus')
# giss_23_precip_hist_aus = convert_PMIP3_to_PRECT_mm_and_get_Aus(giss_23_pr, output_dir, 'giss_23_precip_hist_aus')
# giss_24_precip_hist_aus = convert_PMIP3_to_PRECT_mm_and_get_Aus(giss_24_pr, output_dir, 'giss_24_precip_hist_aus')
# giss_25_precip_hist_aus = convert_PMIP3_to_PRECT_mm_and_get_Aus(giss_25_pr, output_dir, 'giss_25_precip_hist_aus')
# giss_26_precip_hist_aus = convert_PMIP3_to_PRECT_mm_and_get_Aus(giss_26_pr, output_dir, 'giss_26_precip_hist_aus')
# giss_27_precip_hist_aus = convert_PMIP3_to_PRECT_mm_and_get_Aus(giss_27_pr, output_dir, 'giss_27_precip_hist_aus')
# giss_28_precip_hist_aus = convert_PMIP3_to_PRECT_mm_and_get_Aus(giss_28_pr, output_dir, 'giss_28_precip_hist_aus')
                                                           

ipsl_precip_hist_aus = convert_PMIP3_to_PRECT_mm_and_get_Aus(ipsl_pr, output_dir, 'ipsl_precip_hist_aus')
miroc_precip_hist_aus = convert_PMIP3_to_PRECT_mm_and_get_Aus(miroc_pr, output_dir, 'miroc_precip_hist_aus')
mpi_precip_hist_aus = convert_PMIP3_to_PRECT_mm_and_get_Aus(mpi_pr, output_dir, 'mpi_precip_hist_aus')
mri_precip_hist_aus = convert_PMIP3_to_PRECT_mm_and_get_Aus(mri_pr, output_dir, 'mri_precip_hist_aus')


[                                        ] | 0% Completed |  0.0s

/Users/nickywright/miniconda3/envs/py38-x86/lib/python3.8/site-packages/xarray/coding/times.py:144: SerializationWarning: Ambiguous reference date string: 850-01-01 00:00:00. The first value is assumed to be the year hence will be padded with zeros to remove the ambiguity (the padded reference date string is: 0850-01-01 00:00:00). To remove this message, remove the ambiguity by padding your reference date strings with zeros.
  warnings.warn(warning_msg, SerializationWarning)


[########################################] | 100% Completed | 16.4s
[                                        ] | 0% Completed |  0.0s

/Users/nickywright/miniconda3/envs/py38-x86/lib/python3.8/site-packages/xarray/coding/times.py:144: SerializationWarning: Ambiguous reference date string: 850-01-01 00:00:00. The first value is assumed to be the year hence will be padded with zeros to remove the ambiguity (the padded reference date string is: 0850-01-01 00:00:00). To remove this message, remove the ambiguity by padding your reference date strings with zeros.
  warnings.warn(warning_msg, SerializationWarning)


[########################################] | 100% Completed |  7.7s
[                                        ] | 0% Completed |  0.0s

/Users/nickywright/miniconda3/envs/py38-x86/lib/python3.8/site-packages/xarray/coding/times.py:144: SerializationWarning: Ambiguous reference date string: 850-01-01 00:00:00. The first value is assumed to be the year hence will be padded with zeros to remove the ambiguity (the padded reference date string is: 0850-01-01 00:00:00). To remove this message, remove the ambiguity by padding your reference date strings with zeros.
  warnings.warn(warning_msg, SerializationWarning)


[########################################] | 100% Completed | 48.7s
[                                        ] | 0% Completed |  0.0s

/Users/nickywright/miniconda3/envs/py38-x86/lib/python3.8/site-packages/xarray/coding/times.py:144: SerializationWarning: Ambiguous reference date string: 850-01-01 00:00:00. The first value is assumed to be the year hence will be padded with zeros to remove the ambiguity (the padded reference date string is: 0850-01-01 00:00:00). To remove this message, remove the ambiguity by padding your reference date strings with zeros.
  warnings.warn(warning_msg, SerializationWarning)


[########################################] | 100% Completed |  2min 25.7s


In [40]:
hadcm3_precip_hist_aus = convert_PMIP3_to_PRECT_mm_and_get_Aus(hadcm3_pr, output_dir, 'hadcm3_precip_hist_aus')


[########################################] | 100% Completed | 29min 24.7s
